In [ ]:
import robotic as ry
import numpy as np

from time import sleep





def komo_solver(robots, m_objects, goals):
    
    if not isinstance(robots, list):
        robots = [robots]
    if not isinstance(m_objects, list):
        m_objects = [m_objects]
    if not isinstance(goals, list):
        goals = [goals]
    
    komo = ry.KOMO(C, phases=2, slicesPerPhase=20, kOrder=2, enableCollisions=True)  
    komo.addControlObjective([], 0)
    komo.addControlObjective([], 2)
    
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)     
    
    for i in range(len(robots)):
        komo.addObjective([1, -1], ry.FS.distance, [robots[i], m_objects[i]], ry.OT.eq) 
        komo.addModeSwitch([1,-1], ry.SY.stable, [robots[i], m_objects[i]], True) 
        komo.addObjective([2] , ry.FS.positionDiff, [m_objects[i], goals[i]], ry.OT.eq) 

    ret = ry.NLP_Solver(komo.nlp(), verbose=4).solve() 
    
    return (komo, ret)

def komo_onestate_hold(robot, m_object):
    komo = ry.KOMO(C, phases=1, slicesPerPhase=1, kOrder=0, enableCollisions=True)
    komo.addControlObjective([], 0)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq) 
    komo.addObjective([], ry.FS.distance, [robot, m_object], ry.OT.eq)
    
    ret = ry.NLP_Solver(komo.nlp(), verbose= 4).solve()
    return (komo, ret)


def komo_onestate_move(robot, m_object, goal):
    komo = ry.KOMO(C, phases=1, slicesPerPhase=1, kOrder= 1, enableCollisions=True)
    komo.addModeSwitch([0,1], ry.SY.stable, [robot, m_object], True) 
    komo.addObjective([], ry.FS.distance, [robot, m_object], ry.OT.eq)
    komo.addObjective([1, 2] , ry.FS.positionDiff, [m_object, goal], ry.OT.eq) 
    
    ret = ry.NLP_Solver(komo.nlp(), verbose= 4).solve()
    return (komo, ret)

    

def rrt_solver(C, start_state, end_state, trial_count):
    rrt = ry.PathFinder()
    rrt.setProblem(C, start_state, end_state)
    
    ret = rrt.solve()
        
    for i in range(trial_count):
        if ret.feasible:
            return ret
        ret = rrt.solve() 
        print(ret.feasible) 
         
    
    return ret


def rrt_viewer(C, ret, attach_state = None, attach = False):
    
    sleep(1)
    if not ret.feasible:
        return
    
    path = ret.x
    
    initial_state = C.getFrameState()
    
    if attach:
        
        C.setJointState(attach_state)
        C.attach(r1, m1)
        C.attach(r2, m2)
    
    
    for i in path:
        C.setJointState(i)
        C.view()
        sleep(0.03)
    
    C.setFrameState(initial_state)
    
    if attach:
        C.getFrame(m1).unLink()
        C.getFrame(m2).unLink()
        
def path_viewer(C, path, attach_index1, attach_index2):
    
    initial_state = C.getFrameState()
    
    C.view()

    sleep(2)
    
    for i in range(0, len(path)):
        
        
        C.setJointState(path[i])
        C.view()
        sleep(0.05)
        
        if i == attach_index1:
            C.attach(r1, m1)
        if i == attach_index2:
            C.attach(r2, m2)
            
    C.setFrameState(initial_state)
    
    C.getFrame(m1).unLink()
    C.getFrame(m2).unLink()
        
        






def path_mapper(path, final_size):
    original_path, hold_index = path
    original_path = np.array(original_path)
    path_len = len(original_path)

    # Sample evenly spaced indices, reserving one for final state and one for hold
    steps_to_sample = final_size - 2  # 1 for hold, 1 for final
    sample_indices = np.linspace(0, path_len - 2, steps_to_sample).astype(int)
    sampled_path = original_path[sample_indices]

    # Prepare hold and final states
    hold_state = original_path[hold_index].reshape(1, -1)
    final_state = original_path[-1].reshape(1, -1)

    # Determine best insertion point for the hold state
    insert_pos = np.searchsorted(sample_indices, hold_index)

    # Insert hold state at the right spot
    with_hold = np.insert(sampled_path, insert_pos, hold_state, axis=0)

    # Append final state
    final_result = np.vstack([with_hold, final_state])
    return final_result, insert_pos






r1 = "r1"
r2 = "r2"
m1 = "m1"
m2 = "m2"
g1 = "g1"
g2 = "g2"

robots = [r1, r2]
m_objects = [m1, m2]
goals = [g1, g2]

goals_reverse = [g2, g1]


C = ry.Config()

maps = ["map1_simple_cross.g", "map2_tunnel.g", "map3_two_cave.g", "map4_one_cave.g", "map5_pioneer.g"]



C.addFile(f"maps/{maps[4]}")

C.view()





In [ ]:
initial_state = C.getJointState()

first_robot_hold, ret1 = komo_onestate_hold(r1, m1)  
second_robot_hold, ret2 = komo_onestate_hold(r2, m2) 

j1_hold = first_robot_hold.getPath()[0]  # birinci tobotun cargonun yanına geldiği state
j2_hold = second_robot_hold.getPath()[0]      # ikinci tobotun cargonun yanına geldiği state

first_robot_move, ret1 = komo_onestate_move(r1, m1, g2)
second_robot_move, ret2 = komo_onestate_move(r2, m2, g1)

j1_moved = first_robot_move.getPath()[0] # birinci robotun goal yanına geldiği state
j2_moved = second_robot_move.getPath()[0] # ikinci robotun goal yanına geldiği state


inittt = np.concatenate((j1_hold[:3], j2_hold[3:]))
final = np.concatenate((j1_moved[:3], j2_moved[3:]))

C.setJointState(inittt)
C.attach(r1, m1)
C.attach(r2, m2)


rrt_solution = rrt_solver(C, inittt, final, 20)


path_viewer(C, rrt_solution.x, 3, 5)
